In [15]:
from PIL import Image
import numpy as np

In [16]:
x = np.arange(0,32,1) # 绘制这个范围内的直线，使用几个极端的斜率例子：15、75、45度
rate = [
  {'degree':'15', 'rad':np.deg2rad(15), 'color': 'blue', 'code': (0,0,255)},
  {'degree':'45', 'rad':np.deg2rad(45), 'color': 'red', 'code': (255,0,0)},
  {'degree':'75', 'rad':np.deg2rad(75), 'color': 'green', 'code': (0,255,0)},
] # 经过原点的直线

In [17]:
def uniform_draw_line():
  w,h=32,32
  image = Image.new('RGB', (w, h), color='white')
  pixels = image.load()
  for k in rate:
    y = x*np.tan(k['rad'])
    for tx,ty in zip(x,y):
      if 0<=tx<w and 0<=ty<h:
        pixels[tx,h-ty-1] = k['code']
  image.save('uniform.png')
uniform_draw_line()

<!-- ![picture](uniform.png) -->
<img src="uniform.png" alt="示例图片" width="300" height="300">

上面的图片主要反映问题就是直线的绘制过程使用等距取样会导致的问题,在一个相同的区间内跨度大的就会更好看,而其他的就挤到一起了.

In [19]:
def uniform_y_draw_line():
  w,h=32,32
  image = Image.new('RGB', (w, h), color='white')
  pixels = image.load()
  for k in rate:
    y = np.arange(x[0]*k['rad'], x[-1]*k['rad'],1)
    tmp_x = y/k['rad']
    for tx,ty in zip(tmp_x,y):
      if 0<=tx<w and 0<=ty<h:
        pixels[tx,h-ty-1] = k['code']
  image.save('uniform_y.png')
uniform_y_draw_line()

<img src="uniform_y.png" alt="示例图片" width="300" height="300">

上面是y轴的版本，这样的处理只需要作两个轴的版本就能够满足大多数需求。当斜率大于45度就用y，小于用x，此时直线的点的信息是最丰富的。

我想提出的方法是通过等距角度的处理摆除斜率的影响。

In [ ]:
def uniform_deg_draw_line():
  # TODO 这个目测会比较麻烦，涉及到角度转换，下面已经很好用了，我估计直接用下面方法
  w,h=32,32
  image = Image.new('RGB', (w, h), color='white')
  pixels = image.load()
  for k in rate:
    y = np.arange(x[0]*k['rad'], x[-1]*k['rad'],1)
    tmp_x = y/k['rad']
    for tx,ty in zip(tmp_x,y):
      if 0<=tx<w and 0<=ty<h:
        pixels[tx,h-ty-1] = k['code']
  image.save('uniform_deg.png')
uniform_deg_draw_line()

In [51]:
def uniform_45_draw_line():
  w,h=32,32
  image = Image.new('RGB', (w, h), color='white')
  pixels = image.load()
  for k in rate:
    step = 1/np.sqrt(1+k['rad']**2)
    tmp_x = np.arange(x[0], x[-1],step)
    y = np.astype(tmp_x*k['rad'], int)
    tmp_x = np.astype(tmp_x,int)
    t1_x,t1_y,t2_x,t2_y=0,0,0,0
    cnt = 0
    for tx,ty in zip(tmp_x,y):
      if cnt ==0:
        pixels[tx,h-ty-1] = k['code']
        cnt =1
        t2_x,t2_y = tx,ty
      if 0<=tx<w and 0<=ty<h and not \
      (t1_y==t2_y and t2_x==tx \
        or t1_x==t2_x and t2_y==ty):
        pixels[t2_x,h-t2_y-1] = k['code']
        t1_x,t1_y = t2_x,t2_y
        t2_x,t2_y = tx,ty
      elif (t1_y==t2_y and t2_x==tx \
        or t1_x==t2_x and t2_y==ty):
        t2_x,t2_y = tx,ty
    # pixels[tx,h-ty-1] = k['code']
  image.save('uniform_45.png')
uniform_45_draw_line()

上面的方法可以保证连续、宽度为1,取点线段上均匀取点，可用